In [11]:
import torch
from torch import Tensor
from torchvision import transforms
from torch_geometric.nn import GCNConv, global_mean_pool
import torch.nn.functional as F
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from torch_geometric.utils import negative_sampling
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
from tqdm import tqdm

import os
from PIL import Image
from torch.utils.data import Dataset

import torch.nn.functional as F
import numpy as np
from sklearn.feature_extraction import image
import cv2
from typing import Tuple, Optional, Union

import kagglehub

from lib.lib import SignatureDataset, image_to_graph
from torch_geometric.nn import GAE

In [12]:
# Hyperparameters
learning_rate = 1e-3
batch_size = 32  # Changed to match your DataLoader batch_size
epochs = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
class SignatureGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, embedding_dim):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, embedding_dim)
        
    def forward(self, x: Tensor, edge_index: Tensor, batch: Optional[Tensor] = None) -> Tensor:
        """
        x: Node features [num_nodes, in_channels]
        edge_index: Graph edges [2, num_edges]
        batch: Graph IDs for mini-batch training [num_nodes] - not used for GAE
        """
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        x = self.conv3(x, edge_index)
        
        # Return node-level embeddings for GAE
        # Do NOT use global_mean_pool for GAE - it needs node embeddings
        return x  # [num_nodes, embedding_dim]

In [14]:
def dataset_path():
    path = kagglehub.dataset_download("akashgundu/signature-verification-dataset")
    return os.path.join(path, 'extract')

def transform(**kwargs):
    return transforms.Compose([
        transforms.Grayscale(num_output_channels=kwargs['num_output_channels']),
        transforms.Resize(kwargs['resize']),
        transforms.ToTensor(),
    ])

dataset = SignatureDataset(
    root_dir=dataset_path(),
    transform=transform(num_output_channels=1, resize=(150, 150))
)

Loaded 14626 signature images (genuine + forged)


In [15]:
total_size = len(dataset)
train_size = int(0.8 * total_size)
val_size = total_size - train_size
train_dataset, val_dataset = random_split(
    dataset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)
print(f"Dataset sizes - Train: {train_size}, Validation: {val_size}")

Dataset sizes - Train: 11700, Validation: 2926


In [16]:
train_graph = []
val_graph = []

# Convert training dataset
for t in tqdm(train_dataset, desc="Train Graphs"):
    for train_tensor_image in t:
        t_graph = image_to_graph(train_tensor_image)
        train_graph.append(t_graph)

# Convert validation dataset
for v in tqdm(val_dataset, desc="Val Graphs"):
    for val_tensor_image in v:
        v_graph = image_to_graph(val_tensor_image)
        val_graph.append(v_graph)

print("Train graphs:", len(train_graph))
print("Val graphs:", len(val_graph))


Val Graphs: 100%|██████████████████████████████████████████████████████████████████| 2926/2926 [02:11<00:00, 22.32it/s]

Train graphs: 11700
Val graphs: 2926


In [17]:
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.utils import negative_sampling
from datetime import datetime

# Create unique writer for each run
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
writer = SummaryWriter(f'runs/gae_experiment_{timestamp}')

def train_loop(model, graphs, val_graphs, lr, epochs=50, batch=None):
    model = model.to(device)  # move model to GPU/CPU
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-4)
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        
        for graph in tqdm(graphs):
            optimizer.zero_grad()

            # Move graph data to device
            x, edge_index = graph.x.to(device), graph.edge_index.to(device)

            z = model.encode(x, edge_index)
            loss = model.recon_loss(z, edge_index)
            
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        avg_train_loss = total_loss / len(graphs)
        
        # Validation phase
        model.eval()
        total_val_loss, total_auc, total_ap = 0, 0, 0
        
        with torch.no_grad():
            for val_graph in tqdm(val_graphs):
                x, edge_index = val_graph.x.to(device), val_graph.edge_index.to(device)

                z = model.encode(x, edge_index)
                val_loss = model.recon_loss(z, edge_index)
                total_val_loss += val_loss.item()

                # Negative sampling
                pos_edge_index = edge_index
                neg_edge_index = negative_sampling(
                    pos_edge_index, 
                    num_nodes=x.size(0),
                    num_neg_samples=pos_edge_index.size(1)
                ).to(device)

                # Predictions
                pos_pred = model.decoder(z, pos_edge_index, sigmoid=True)
                neg_pred = model.decoder(z, neg_edge_index, sigmoid=True)

                # Labels + predictions
                y_true = torch.cat([
                    torch.ones(pos_pred.size(0), device=device),
                    torch.zeros(neg_pred.size(0), device=device)
                ])
                y_pred = torch.cat([pos_pred, neg_pred])

                # Metrics (move to CPU for sklearn)
                auc = roc_auc_score(y_true.cpu(), y_pred.cpu())
                ap = average_precision_score(y_true.cpu(), y_pred.cpu())
                
                total_auc += auc
                total_ap += ap
                
        avg_val_loss = total_val_loss / len(val_graphs)
        avg_auc = total_auc / len(val_graphs)
        avg_ap = total_ap / len(val_graphs)
        
        # Log
        writer.add_scalar('Accuracy/AUC', avg_auc, epoch)
        writer.add_scalar('Accuracy/AP', avg_ap, epoch)
        writer.add_scalar('Loss/Train', avg_train_loss, epoch)
        writer.add_scalar('Loss/Val', avg_val_loss, epoch)
        
        print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, AUC: {avg_auc:.4f}, AP: {avg_ap:.4f}')
    
    writer.close()
    return model

In [19]:
# from torch_geometric.nn import VGAE
input_dim = next(iter(train_graph)).x.shape[1]
hidden_dim = 64
embedding_dim = 128
epochs = 500
# epochs = 200

encoder = SignatureGNN(input_dim, hidden_dim, embedding_dim)
model = GAE(encoder)

try:
    gae = train_loop(model, train_graph, val_graph, learning_rate, epochs)
except KeyboardInterrupt:
    print("Saving Model...")
    torch.save(model.state_dict(), "VAE_GNN_model.pth")
    

  2%|█▎                                                                            | 203/11700 [00:09<09:09, 20.94it/s]


Saving Model...


NameError: name 'orch' is not defined

In [ ]:
torch.save(gae.state_dict(), "VAE_GNN_model.pth")

In [ ]:
z = next(iter(train_graph))
print(z)

In [ ]:
def transform(**kwargs):
    return transforms.Compose([
        transforms.Grayscale(num_output_channels=kwargs['num_output_channels']),
        transforms.Resize(kwargs['resize']),
        transforms.ToTensor(),
    ])

dataset = SignatureDataset(
    root_dir="test_image",
    transform=transform(num_output_channels=1, resize=(150, 150))
)

test_graph = []

for t in dataset:
    for test_tensor_image in t:
        t_graph = image_to_graph(test_tensor_image)
        test_graph.append(t_graph)

graphs = DataLoader(test_graph, batch_size=batch_size, shuffle=False)
print(len(test_graph))

In [170]:
encoder = SignatureGNN(input_dim, hidden_dim, embedding_dim)
model = GAE(encoder)
model.load_state_dict(torch.load('model.pth'))
model.eval()

with torch.no_grad():
    z = model.encode(sample.x, sample.edge_index)

In [171]:
z

tensor([[ 0.1388, -0.1934,  0.1532,  ...,  0.1042,  0.1565,  0.0883],
        [ 0.1617, -0.2071,  0.1523,  ...,  0.1083,  0.1765,  0.0973],
        [ 0.1558, -0.2031,  0.1537,  ...,  0.1067,  0.1720,  0.0956],
        ...,
        [-0.0241,  0.0962, -0.0785,  ..., -0.0100, -0.1637, -0.1377],
        [ 0.0465,  0.0573, -0.0675,  ...,  0.0338, -0.0884, -0.1084],
        [ 0.0884,  0.0128, -0.0262,  ...,  0.0715, -0.0210, -0.0687]])